## 청소년 수련시설

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import trange, tqdm, tqdm_notebook

In [3]:
grid = gpd.read_file('data/bucheongrid/bucheonjsonjson.geojson')
    

In [11]:
bike_bucheon = pd.read_csv('output/bike_bucheon.csv')
bike_bucheon = bike_bucheon.iloc[:, range(1,4)]
display(bike_bucheon)

,자전거대여소명,위도,경도
0,공공자전거대여소 부천시청역,37.504938,126.763465
1,공공자전거대여소 상동역,37.506069,126.754122
2,공공자전거대여소 신중동역,37.503068,126.775751
3,공공자전거대여소 부천역,37.484717,126.782677
4,공공자전거대여소 송내역,37.488286,126.755075
5,공공자전거대여소 부천시청,37.503728,126.766102


In [13]:
teenage_facil = pd.read_csv('output/teenage_facil.csv')
teenage_facil = teenage_facil.iloc[:, range(1,5)]
teenage_facil.columns = ["시설명","소재지","위도","경도"]
display(teenage_facil)

,시설명,소재지,위도,경도
0,부천시청소년수련관,장말로 107 (복사골문화센터 2층),37.494783,126.754214
1,산울림청소년수련관,역곡로 149-65(춘의동),37.498421,126.804987
2,소사청소년수련관,소사동로 125(괴안동),37.476104,126.807489
3,부천여성청소년센터,부천로122번길 60(원미동),37.494498,126.788533
4,고리울청소년문화의집,고강로132번길 28(고강동),37.530065,126.818651
5,송내동청소년문화의집,경인로92번길 33(송내어울마당 3층),37.482505,126.766558


In [7]:
temp = gpd.read_file('data/bucheongrid/bucheonjsonjson.geojson')
temp = temp.dropna(axis=0)
축척 = sum([
    *(temp['geometry'].bounds['maxx'] - temp['geometry'].bounds['minx']),
    *(temp['geometry'].bounds['maxy'] - temp['geometry'].bounds['miny']),
    ]) / (2 * len(temp)) / 100
도보속도 = 133.1

In [8]:
minute = 5
거리 = minute * 133.1 / 100 * 60

In [9]:
def multipolygon_to_경도(x):
    경도, 위도 = x[0].exterior.xy 
    return np.array([x for x in 경도])
def multipolygon_to_위도(x):
    경도, 위도 = x[0].exterior.xy 
    return np.array([x for x in 위도])

In [14]:
def get_within_nminutes_infor(Data_input, Data_infor_input, name, value=False, geojson=False):
    """
    array을 이용하여 한번에 거리를 계산 후, 도보 3분이내 거리안에 해당하는 경우의
    수, 거리의 최소,평균,최대, 값의 최소,평균,최대를 계산합니다.
    
    geojson파일인 경우 멀티폴리곤에서 각각 경위도를 계산해주고 작업을 시행합니다.
    """
    
    Data = Data_input.copy()
    Data_infor = Data_infor_input.copy()
    
    print(name + "_작업중.............")
    
    ## geojson의 경우.. 지적도의 경우 경위도를 만들어 넣어 줄 거기 때문에 try로 함수입력
    if geojson:
        try:
            Data_infor['경도'] = Data_infor['geometry'].apply(multipolygon_to_경도) 
            Data_infor['위도'] = Data_infor['geometry'].apply(multipolygon_to_위도)        
        except:
            try:
                Data_infor['경도'] = Data_infor['geometry'].centroid.x
                Data_infor['위도'] = Data_infor['geometry'].centroid.y
            except:
                pass
            
    Len1 = len(Data)
    Len2 = len(Data_infor)
    
    tt_위도 = np.repeat(Data['위도'], Len2)
    tt_위도 = np.array(tt_위도).reshape(Len1, Len2)
    diff_위도 = tt_위도 - np.array(Data_infor['위도'])

    tt_경도 = np.repeat(Data['경도'], Len2)
    tt_경도 = np.array(tt_경도).reshape(Len1, Len2)
    diff_경도 = tt_경도 - np.array(Data_infor['경도'])
    
    diff_위도 = diff_위도**2
    diff_경도 = diff_경도**2
    dist = (diff_위도 + diff_경도)**(0.5)
    
    
    Count = []
    Dist_min = []
    Dist_mean = []
    Dist_max = []
    
    Val_min = []
    Val_mean = []
    Val_max = []
    
    
    
    for idx in tqdm(Data.index):
        if geojson:
            dist_idx = np.array([i.min() for i in dist[idx]])
        else:
            dist_idx = dist[idx]
        
        temp = Data_infor.loc[dist_idx < 거리 * 축척]           
        Count.append(len(temp))
        
        try:
            Dist_min.append(dist_idx[dist_idx < 거리 * 축척].min())
            Dist_mean.append(dist_idx[dist_idx < 거리 * 축척].mean())
            Dist_max.append(dist_idx[dist_idx < 거리 * 축척].max())
        except:
            ## 수정된 부분!
            Dist_min.append(0.0025)
            Dist_mean.append(0.0025)
            Dist_max.append(0.0025)
            
    Data[name + "_" + str(minute) + "분이내수"] = Count
    Data[name + "_" + str(minute) + "분이내거리최소"] = Dist_min
    Data[name + "_" + str(minute) + "분이내거리평균"] = Dist_mean
    Data[name + "_" + str(minute) + "분이내거리최대"] = Dist_max
        
    
    if value:
        for idx in tqdm(Data.index):
            if geojson:
                dist_idx = np.array([i.min() for i in dist[idx]])
            else:
                dist_idx = dist[idx]
                
            temp = Data_infor.loc[dist_idx < 거리 * 축척]
            Val_min.append(temp[value].min())
            Val_mean.append(temp[value].mean())
            Val_max.append(temp[value].max())
            
        Data[name + "_" + str(minute) + "분이내값최소"] = Val_min
        Data[name + "_" + str(minute) + "분이내값평균"] = Val_mean
        Data[name + "_" + str(minute) + "분이내값최대"] = Val_max
        Data = Data.fillna(0)
        
        return Data
    
    else:
        return Data

In [15]:
s = get_within_nminutes_infor(Data_input = bike_bucheon, Data_infor_input = teenage_facil, name="청소년 수련시설")

청소년 수련시설_작업중.............


100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 601.66it/s]


In [16]:
s

,자전거대여소명,위도,경도,청소년 수련시설_5분이내수,청소년 수련시설_5분이내거리최소,청소년 수련시설_5분이내거리평균,청소년 수련시설_5분이내거리최대
0,공공자전거대여소 부천시청역,37.504938,126.763465,0,0.0025,0.0025,0.0025
1,공공자전거대여소 상동역,37.506069,126.754122,0,0.0025,0.0025,0.0025
2,공공자전거대여소 신중동역,37.503068,126.775751,0,0.0025,0.0025,0.0025
3,공공자전거대여소 부천역,37.484717,126.782677,0,0.0025,0.0025,0.0025
4,공공자전거대여소 송내역,37.488286,126.755075,0,0.0025,0.0025,0.0025
5,공공자전거대여소 부천시청,37.503728,126.766102,0,0.0025,0.0025,0.0025


In [17]:
s.to_csv("output/var_teenagefacil.csv")

In [20]:
starbucks = pd.read_csv('output/starbucks.csv')
starbucks = starbucks.iloc[:, range(1,5)]
starbucks.columns = ["시설명","소재지","위도","경도"]
display(starbucks)

,시설명,소재지,위도,경도
0,신중동,경기도 부천시 신흥로 187,37.500323,126.776481
1,춘의역DT,경기도 부천시 길주로 386,37.503080,126.785394
2,부천중동센트럴,경기도 부천시 소향로 181,37.502070,126.767928
3,부천상동,경기도 부천시 상동로 87,37.505223,126.752415
4,부천,경기도 부천시 부천로 7,37.485069,126.782096
5,송내역,경기도 부천시 부일로 204,37.488345,126.752603
6,부천플레이도시,경기도 부천시 조마루로 2,37.499970,126.744235
7,현대중동,경기도 부천시 길주로 180,37.504318,126.762084
8,부천시청역,경기도 부천시 길주로 181,37.505314,126.762634
9,부천중동,경기도 부천시 신흥로 161,37.498002,126.777043


In [21]:
s = get_within_nminutes_infor(Data_input = bike_bucheon, Data_infor_input = starbucks, name="스타벅스")

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 547.87it/s]

스타벅스_작업중.............


In [22]:
s

,자전거대여소명,위도,경도,스타벅스_5분이내수,스타벅스_5분이내거리최소,스타벅스_5분이내거리평균,스타벅스_5분이내거리최대
0,공공자전거대여소 부천시청역,37.504938,126.763465,3,0.000912,0.001141,0.001514
1,공공자전거대여소 상동역,37.506069,126.754122,3,0.000322,0.001816,0.003222
2,공공자전거대여소 신중동역,37.503068,126.775751,2,0.001757,0.002299,0.002840
3,공공자전거대여소 부천역,37.484717,126.782677,1,0.000679,0.000679,0.000679
4,공공자전거대여소 송내역,37.488286,126.755075,1,0.002473,0.002473,0.002473
5,공공자전거대여소 부천시청,37.503728,126.766102,4,0.002179,0.003130,0.004061


In [23]:
s.to_csv("output/var_starbucks.csv")